# Overview

This notebook provides an example on how to use the provided code to load connectomes and perform Connectome Spatial Smoothing (CSS) at different connectome resolutions.

For a better interactive viewing experience, you could open this notebook with google colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sina-mansour/connectome-based-smoothing/blob/main/notebooks/example.ipynb)

The codes for high-resolution connectome mapping are adopted from [this repository](https://github.com/sina-mansour/neural-identity). If you are using the high-resolution connectome mapping tools, please make sure to cite the original article too:

**Mansour, L. Sina, et al. "High-resolution connectomic fingerprints: mapping neural identity and behavior." *NeuroImage* (2021): 117695.**


## Importing packages

Cells in this section import the required python packages:

Package installations (for google colab only)

In [ ]:
# install package on colab
! pip install Connectome-Spatial-Smoothing


Package imports

In [1]:
# First, import basic python packages
import os
import sys
import numpy as np

# Additionally, import the CSS package
from Connectome_Spatial_Smoothing import CSS as css


## Loading files

Cells in this section present the neuroimaging data and explain how to load the required files.

A set of sample imaging files are included in `data` which are used in this notebook to provide examples of using the code.

Here's a brief explanation of all files loaded in this example to map the connectomes and perform CSS:

- **Tractography file:** A tractography file (`sample_tractography_10k.tck`) was generated using MRtrix3. For the purpose of this example the file only contains 10,000 streamlines to reduce all computational and storage requirements. This is a sagital view of what this file contains when visualized with Mrtrix's mrview:

<img src="../static/images/sample_tractography.png" alt="Tractography" style="width: 500px;"/>

- **Native surface:** The left and righ native white-matter surface meshes (32k fs-LR) (`sample.native.L.white.32k_fs_LR.surf.gii` &`sample.native.R.white.32k_fs_LR.surf.gii`) are used to map the connectomes. This is what the left surface looks like:

<img src="../static/images/sample_native_surface.png" alt="Tractography" style="width: 500px;"/>

- **MNI surface:** Additionally left and righ white-matter surface meshes (32k fs-LR) in MNI152 space are provided. These can be used in combination with the nonlinear warp to directly map connectomes in the standard space. This is what the left surface warped to MNI looks like:

<img src="../static/images/sample_MNI152_surface.png" alt="Tractography" style="width: 500px;"/>

- **Warp file:** The nonlinear warp file (`sample_standard2acpc_dc.nii.gz`) is also required to map the connectome directly onto standard space surface mesh.

- **Brain atlas:** To map the connectomes to an atlas resolution we use the [HCP MMP1.0 brain atlas](https://www.nature.com/articles/nature18933):

<img src="../static/images/glasser_atlas.png" alt="Tractography" style="width: 500px;"/>


In [2]:
# Get paths for all files:
main_dir = os.path.abspath(os.path.dirname(css.__file__))

tractography_file = os.path.abspath('{}/data/sample/sample_tractography_10k.tck'.format(main_dir))

left_native_surface_file = os.path.abspath('{}/data/sample/sample.native.L.white.32k_fs_LR.surf.gii'.format(main_dir))
right_native_surface_file = os.path.abspath('{}/data/sample/sample.native.R.white.32k_fs_LR.surf.gii'.format(main_dir))

left_MNI_surface_file = os.path.abspath('{}/data/sample/sample.MNI152.L.white.32k_fs_LR.surf.gii'.format(main_dir))
right_MNI_surface_file = os.path.abspath('{}/data/sample/sample.MNI152.R.white.32k_fs_LR.surf.gii'.format(main_dir))

warp_file = os.path.abspath('{}/data/sample/sample_standard2acpc_dc.nii.gz'.format(main_dir))

brain_atlas_file = os.path.abspath('{}/data/templates/atlas/Glasser360.32k_fs_LR.dlabel.nii'.format(main_dir))


## Mapping the connectomes

Cells in this section explain how to map connectomes from the loaded files:

### High-resolution connectomes

Cells in this section can be used to map high-resolution structural connectomes from tractography data:

The following function can be used to map the high-resolution structural connectome onto the native surface_mesh:

In [3]:
# Map high-resolution connectome onto native surfaces:
native_high_resolution_connectome = css.map_high_resolution_structural_connectivity(tractography_file, left_native_surface_file, right_native_surface_file)


Sun Nov 28 19:47:37 2021: [INFO] loading tractography file.
Sun Nov 28 19:47:37 2021: [INFO] track file loaded: /home/sina/.local/lib/python3.8/site-packages/Connectome_Spatial_Smoothing/data/sample/sample_tractography_10k.tck
Sun Nov 28 19:47:37 2021: [INFO] endpoints extracted: #10000
Sun Nov 28 19:47:38 2021: [INFO] closest brainordinates located
Sun Nov 28 19:47:38 2021: [INFO] outliers located: #3027 outliers (30.27%, with threshold 2mm)
Sun Nov 28 19:47:38 2021: [INFO] creating sparse incidence matrix
Sun Nov 28 19:47:38 2021: [INFO] sparse matrix generated


Alternatively, we could map the connectomes directly onto the MNI surface by providing an additional nonlinear warp:

In [4]:
# Map high-resolution connectome onto native surfaces:
high_resolution_connectome = css.map_high_resolution_structural_connectivity(tractography_file, left_MNI_surface_file, right_MNI_surface_file, warp_file=warp_file)


Sun Nov 28 19:47:39 2021: [INFO] loading tractography file.
Sun Nov 28 19:47:39 2021: [INFO] track file loaded: /home/sina/.local/lib/python3.8/site-packages/Connectome_Spatial_Smoothing/data/sample/sample_tractography_10k.tck
Sun Nov 28 19:47:39 2021: [INFO] endpoints extracted: #10000
Sun Nov 28 19:47:39 2021: [INFO] endpoints warped: #10000
Sun Nov 28 19:47:42 2021: [INFO] closest brainordinates located
Sun Nov 28 19:47:42 2021: [INFO] outliers located: #3532 outliers (35.32%, with threshold 2mm)
Sun Nov 28 19:47:42 2021: [INFO] creating sparse incidence matrix
Sun Nov 28 19:47:42 2021: [INFO] sparse matrix generated


It's important to note that the connectivity matrices are loaded in a sparce matrix format. (This ensurers effiecient matrix manipulations when performing CBS in high-resolution.

In [5]:
# Check the output matrix
high_resolution_connectome


<59412x59412 sparse matrix of type '<class 'numpy.float32'>'
	with 12913 stored elements in Compressed Sparse Row format>

The connectome mapping code above also uses a threshold (default: 2mm) to exclude any streamlines ending far from the cortical surface vertices. You may look at the function's help for further details:

In [6]:
help(css.map_high_resolution_structural_connectivity)


Help on function map_high_resolution_structural_connectivity in module Connectome_Spatial_Smoothing.CSS:

map_high_resolution_structural_connectivity(track_file, left_surface_file, right_surface_file, warp_file=None, threshold=2, subcortex=False)
    Map the high-resolution structural connectivity matrix from tractography outputs.
    
    Args:
    
        track_file: The tractography file to map connectivity from (tck format)
    
        left_surface_file: The left hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        right_surface_file: The right hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        warp_file: [optional] A nonlinear warp can be provided to map streamlines after warping
                   the endpoints.
    
        threshold: [default=2] A threshold to exclude endpoints further than that threshold from
                   any cortical vertices (in mm).
    
    Returns:
    
        connectome: The high-resolution structural connec

### Atlas-resolution connectomes

Cells in this section can be used to map structural connectomes at the resolution of a brain atlas:

#### Loading the brain atlas

We have provided a function to get the $p \times v$ representation of the brain atlas $P$ along with it's labels. (Check our paper for further detail)

In [7]:
labels, parcellation_matrix = css.parcellation_characteristic_matrix(atlas_file=brain_atlas_file)


#### Mapping parcellation connectome

Using the matrix representation of the brain atlas $P$. The high-resolution connectivity matrix $A$ can be downsampled to a parcellation connectome $A_p$. (Check our paper for further detail.

$A_p = PAP^T$

In [8]:
atlas_connectome = css.downsample_high_resolution_structural_connectivity_to_atlas(high_resolution_connectome, parcellation_matrix)


In [9]:
# check output connectome's dimension
atlas_connectome.shape

(360, 360)

Alternatively you could use the following function to directly map atlas connectomes from tractography files:

In [10]:
# Map high-resolution connectome onto native surfaces:
atlas_connectome = css.map_atlas_resolution_structural_connectivity(tractography_file, left_MNI_surface_file, right_MNI_surface_file, atlas_file=brain_atlas_file, warp_file=warp_file)


Sun Nov 28 19:48:14 2021: [INFO] loading tractography file.
Sun Nov 28 19:48:14 2021: [INFO] track file loaded: /home/sina/.local/lib/python3.8/site-packages/Connectome_Spatial_Smoothing/data/sample/sample_tractography_10k.tck
Sun Nov 28 19:48:14 2021: [INFO] endpoints extracted: #10000
Sun Nov 28 19:48:14 2021: [INFO] endpoints warped: #10000
Sun Nov 28 19:48:15 2021: [INFO] closest brainordinates located
Sun Nov 28 19:48:15 2021: [INFO] outliers located: #3532 outliers (35.32%, with threshold 2mm)
Sun Nov 28 19:48:15 2021: [INFO] creating sparse incidence matrix
Sun Nov 28 19:48:15 2021: [INFO] sparse matrix generated


Make sure to check the functions' docstrings for further detail:

In [11]:
help(css.parcellation_characteristic_matrix)


Help on function parcellation_characteristic_matrix in module Connectome_Spatial_Smoothing.CSS:

parcellation_characteristic_matrix(atlas_file='/home/sina/.local/lib/python3.8/site-packages/Connectome_Spatial_Smoothing/data/templates/atlas/Glasser360.32k_fs_LR.dlabel.nii')
    This function generates a p x v characteristic matrix from a brain atlas.
    
    Args:
    
        atlas_file: path to a cifti atlas file (dlabel.nii) [default: HCP MMP1.0]
    
    Returns:
    
        parcellation_matrix: The p x v sparse matrix representation of the atlas.



In [12]:
help(css.downsample_high_resolution_structural_connectivity_to_atlas)


Help on function downsample_high_resolution_structural_connectivity_to_atlas in module Connectome_Spatial_Smoothing.CSS:

downsample_high_resolution_structural_connectivity_to_atlas(high_resolution_connectome, parcellation)
    Downsample the high-resolution structural connectivity matrix to the resolution of a brain atlas.
    
    Args:
    
        high_resolution_connectome: The high-resolution structural connectome (59412 x 59412 sparse CSR matrix)
    
        parcellation: A p x v sparse percellation matrix (can also accept a soft parcellation)
    
    Returns:
    
        connectome: The atlas-resolution structural connectome in a sparse csr format.



In [13]:
help(css.map_atlas_resolution_structural_connectivity)


Help on function map_atlas_resolution_structural_connectivity in module Connectome_Spatial_Smoothing.CSS:

map_atlas_resolution_structural_connectivity(track_file, left_surface_file, right_surface_file, atlas_file='/home/sina/.local/lib/python3.8/site-packages/Connectome_Spatial_Smoothing/data/templates/atlas/Glasser360.32k_fs_LR.dlabel.nii', warp_file=None, threshold=2, subcortex=False)
    Maps the structural connectivity matrix at the resolution of a brain atlas.
    
    Args:
    
        track_file: The tractography file to map connectivity from (tck format)
    
        left_surface_file: The left hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        right_surface_file: The right hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        warp_file: [optional] A nonlinear warp can be provided to map streamlines after warping
                   the endpoints.
    
        threshold: [default=2] A threshold to exclude endpoints further than that thresho

## Connectome Spatial Smoothing

Cells in this section show how to use the codes to perform CSS:

### CSS smoothing kernel

The CSS smoothing kernel $F_s$ is computed by column normalization of a Gaussian smoothing kernel $F_G$ which is computed based on the geodesic distances on the surface meshes.

The following function can be used to compute the CSS smoothing kernel from **FWHM** and $\varepsilon$: (the function takes a while to run)


In [14]:
%%time
smoothing_kernel = css.compute_smoothing_kernel(left_MNI_surface_file, right_MNI_surface_file, fwhm=3, epsilon=0.01)


CPU times: user 851 ms, sys: 61.4 ms, total: 912 ms
Wall time: 1min 25s


In [15]:
help(css.compute_smoothing_kernel)

Help on function compute_smoothing_kernel in module Connectome_Spatial_Smoothing.CSS:

compute_smoothing_kernel(left_surface_file, right_surface_file, fwhm, epsilon=0.01)
    Compute the CSS smoothing kernel.
    
    Args:
    
        left_surface_file: The left hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        right_surface_file: The right hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        fwhm: The full width at half maximum (FWHM) in mm.
    
        epsilon: The kernel truncation threshold.
    
    Returns:
    
        kernel: The v x v high-resolution smoothing kernel.



### High-resolution CSS

We could then use the computed smoothing kernel to perform CSS on a high-resolution connectome:

$A_s = F_s A {F_s}^T$

In [16]:
smoothed_high_resolution_connectome = css.smooth_high_resolution_connectome(high_resolution_connectome, smoothing_kernel)


### Atlas-resolution CSS

To smooth the connectomes in atlas resolution, we need to first compute a $p \times v$ smoothed soft parcellation $P_s$. This soft parcellation can be computed from the brain parcellation matrix $P$ and the CSS smoothing kernel $F_s$:

$P_s = P F_s$

In [17]:
smoothed_parcellation = css.smooth_parcellation_matrix(parcellation_matrix, smoothing_kernel)


This soft parcellation can now be used to produce CBS smoothed atlas connectomes:

In [18]:
smoothed_atlas_connectome = css.downsample_high_resolution_structural_connectivity_to_atlas(high_resolution_connectome, smoothed_parcellation)
